## Create Filepaths

In [40]:
from pprint import pprint
FPATHS = dict(

    Data={
        "ml":{
            "train":"Data/training_data_ml.joblib",
            "test":"Data/test_data_ml.joblib",
            "target_lookup":"Data/target_lookup.joblib",
            "label_encoder":"Data/label_encoder.joblib",
         },
        "tf":{
            "train_tf":"Data/train_ds.joblib",
            "test_tf":"Data/test_ds.joblib",
        },
    },

    Models={
        "ml":"Models/Gridsearch_full.joblib",
        "grid":"Models/gridsearch_89.joblib",
        "neural":"Models/hybrid.joblib",
    }
)
pprint(FPATHS)


{'Data': {'ml': {'label_encoder': 'Data/label_encoder.joblib',
                 'target_lookup': 'Data/target_lookup.joblib',
                 'test': 'Data/test_data_ml.joblib',
                 'train': 'Data/training_data_ml.joblib'},
          'tf': {'test_tf': 'Data/test_ds.joblib',
                 'train_tf': 'Data/train_ds.joblib'}},
 'Models': {'grid': 'Models/gridsearch_89.joblib',
            'ml': 'Models/Gridsearch_full.joblib',
            'neural': 'Models/hybrid.joblib'}}


## Imports

In [62]:
# Save the filepaths in a config folder
import os, json
import sys
os.makedirs('config/', exist_ok=True)
FPATHS_FILE = 'config/filepaths.json'
with open(FPATHS_FILE, 'w') as f:
    json.dump(FPATHS, f)



In [42]:
import pandas as pd
import joblib
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
import spacy
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import ConfusionMatrixDisplay, classification_report
import tensorflow as tf
# Increase column width
pd.set_option('display.max_colwidth', 250)
import tensorflow as tf


In [43]:
%load_ext autoreload 
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
# Using function from Creating a File Structure Lesson
fn.create_directories_from_paths(FPATHS)


## Save Training and Test Data

In [15]:
df = joblib.load('Data-NLP/df_high_low_reviews-1')
df.head()

,review_id,movie_id,imdb_id,original_title,review,rating,tokens,joined_tokens,lemmas,joined_lemmas
6,57b2d23dc3a36837d7000b14,955,tt0120755,Mission: Impossible II,"The first underwhelmed me, but this one straight-up bored me. Again, of course seeing Hunt climb a mountain without a harness is impressive sure. And I even quite liked the idea behind the villain of the piece (though even that angle was woefully...",3.0,"[underwhelmed, straight, bored, course, seeing, hunt, climb, mountain, harness, impressive, sure, liked, idea, villain, piece, angle, woefully, underdeveloped, setting, predominantly, australia, grab, attention, saying, cause, pretty, biased, com...",underwhelmed straight bored course seeing hunt climb mountain harness impressive sure liked idea villain piece angle woefully underdeveloped setting predominantly australia grab attention saying cause pretty biased comes hear tell mission impossi...,"[underwhelme, straight, bore, course, see, hunt, climb, mountain, harness, impressive, sure, like, idea, villain, piece, angle, woefully, underdeveloped, set, predominantly, australia, grab, attention, say, cause, pretty, biased, come, hear, tell...",underwhelme straight bore course see hunt climb mountain harness impressive sure like idea villain piece angle woefully underdeveloped set predominantly australia grab attention say cause pretty biased come hear tell mission impossible ii low poi...
7,5e9120963679a10018375490,955,tt0120755,Mission: Impossible II,After quite entertainign Mission Impossible I the second installment turned out ... terrible. As if the screenwriters didn't know how to fill the 2 hrs with action the overuse of heroic slow motion scenes is horrible. You almost might need a barf...,2.0,"[entertainign, mission, impossible, second, installment, turned, terrible, screenwriters, know, fill, 2, hrs, action, overuse, heroic, slow, motion, scenes, horrible, need, barf, bag, stand, slo, mo, seconds, 15, seconds, amazed, decide, push, m,...",entertainign mission impossible second installment turned terrible screenwriters know fill 2 hrs action overuse heroic slow motion scenes horrible need barf bag stand slo mo seconds 15 seconds amazed decide push m movies turned better hard beat l...,"[entertainign, mission, impossible, second, installment, turn, terrible, screenwriter, know, fill, 2, hrs, action, overuse, heroic, slow, motion, scene, horrible, need, barf, bag, stand, slo, mo, second, 15, second, amazed, decide, push, m, movie...",entertainign mission impossible second installment turn terrible screenwriter know fill 2 hrs action overuse heroic slow motion scene horrible need barf bag stand slo mo second 15 second amazed decide push m movie turn well hard beat lousy piece ...
20,6409202f0e441900e887a17b,4234,tt0134084,Scream 3,**_Scream 3’s_ lackluster screenplay and unimaginative kills leave a film that is a bore to watch.**\r\n\r\nThe meta-narrative of trilogies throughout the film does not make up for how abysmal the plot was. This film creates so much lore for the ...,4.0,"[scream, 3, lackluster, screenplay, unimaginative, kills, leave, film, bore, watch, meta, narrative, trilogies, film, abysmal, plot, film, creates, lore, past, movies, seemingly, blue, muddling, continuity, shock, value, added, little, film, felt...",scream 3 lackluster screenplay unimaginative kills leave film bore watch meta narrative trilogies film abysmal plot film creates lore past movies seemingly blue muddling continuity shock value added little film felt forced entire plot point sidne...,"[scream, 3, lackluster, screenplay, unimaginative, kill, leave, film, bore, watch, meta, narrative, trilogy, film, abysmal, plot, film, create, lore, past, movie, seemingly, blue, muddle, continuity, shock, value, add, little, film, feel, force, ...",scream 3 lackluster screenplay unimaginative kill leave film bore watch meta narrative trilogy film abysmal plot film create lore past movie seemingly blue muddle continuity shock value add littl

In [16]:
df_ml = df

In [18]:
def create_groups(x):
    if x<=4.0:
        return 0
    elif x >=5.0:
        return 1
    else: 
        return None

In [19]:
## Use the function to create a new "rating" column with groups
df_ml['high_low'] = df_ml['rating'].map(create_groups)
df_ml['high_low'].value_counts(dropna=False)

0    1224
1    1195
Name: high_low, dtype: int64

In [20]:
# Mapping numeric value with public-facing label
rename_target_map = {0:'Bad Review', 1:"Good Review"}

In [21]:
# Define X and y
X = df_ml['review']
y= df_ml["high_low"].replace(rename_target_map)

In [22]:
# Perform 70:30 train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42)
len(X_train), len(X_test)

(1693, 726)

In [24]:
# Getting unique classes to convert 
class_names = y_train.unique()
class_names


array(['Good Review', 'Bad Review'], dtype=object)

In [25]:
from sklearn.preprocessing import LabelEncoder
# Fit the label encoder on unique class names
encoder = LabelEncoder()
encoder.fit(class_names)

LabelEncoder()

In [27]:
# Transform the y_train_str and y_test_str with encoder
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)
y_train[:10]


array([1, 0, 0, 0, 1, 1, 0, 0, 0, 0])

In [30]:
# Extract the class names from the encoder
classes = encoder.classes_
classes


array(['Bad Review', 'Good Review'], dtype=object)

In [31]:
# Get the encoded values for each both classes
class_codes = encoder.transform(classes)
class_codes


array([0, 1])

In [32]:
# Making lookup dictionary to find the encoded label's original name
target_lookup = dict(zip(class_codes,classes ))
target_lookup


{0: 'Bad Review', 1: 'Good Review'}

In [37]:

fpath_train  = FPATHS['Data']['ml']['train'] # Save training data
joblib.dump([X_train, y_train], fpath_train)


fpath_test = FPATHS['Data']['ml']['test'] # Save testing data
joblib.dump([X_test, y_test], fpath_test)


['Data/test_data_ml.joblib']

In [39]:

fpath_encoder = FPATHS['Data']['ml']['label_encoder'] # Save encoder
joblib.dump(encoder,fpath_encoder)


fpath_lookup = FPATHS['Data']['ml']['target_lookup'] # Save dictionary
joblib.dump(target_lookup,fpath_lookup )


['Data/target_lookup.joblib']

## Deep NLP - Tensorflow

In [46]:
# load training data using dictionary
[X_train, y_train] = joblib.load(FPATHS['Data']['ml']['train'])
[X_test, y_test]  = joblib.load(FPATHS['Data']['ml']['test'])
X_train[:5],y_train[:5]

(4788    I saw this movie last year when it came out. I thought it was very cool and a nice take on the superhero origin story, along with "young black man makes it out of the ghetto" story.  Not very believable towards the end, but the acting and the rul...
 7735    I was expecting more from this movie, but it turned out to be a big disappointment.\r\n\r\nFirst of all Vikings in this movie were represented more like an animals, acting like a rabid animals rather than humans and Vikings we have seen in popula...
 3548                                                                                                                                                                                            Christian propaganda trash. An embarrassment to the word 'movie'.
 7663    Where has the "magic" gone? Disney muddles the mix once again and ruins a childhood favorite by removing the accomplished Buzz frontman Tim Allen for Chris Evans? I was so wanting to see the quick quips and humor a

In [47]:
# Make a dataset object from train data
train_tf = tf.data.Dataset.from_tensor_slices((X_train, y_train))
# Make a dataset object from test data
test_tf = tf.data.Dataset.from_tensor_slices((X_test, y_test))


In [48]:
# Shuffle dataset
train_tf = train_tf.shuffle(buffer_size=len(train_tf),reshuffle_each_iteration=False)


In [50]:
# Set the ratio of the train, validation, test split
split_train = .75
split_val =  .25
# Calculate the number of samples for training and validation data 
n_train_samples =  int(len(train_tf) * split_train)
n_val_samples = int(len(train_tf) * split_val)
# Set the batch size
BATCH_SIZE =32
import math
# math.ceil will round up
# How many batches? 
n_train_batches = math.ceil(n_train_samples/BATCH_SIZE)
n_val_batches = math.ceil(n_val_samples/BATCH_SIZE)
print(f"    - train:\t{n_train_samples} samples \t({n_train_batches} batches)")
print(f"    - val:  \t{n_val_samples} samples \t({n_val_batches} batches)")


    - train:	1269 samples 	(40 batches)
    - val:  	423 samples 	(14 batches)


In [51]:
# Use take and skip to define each set
train_ds = train_tf.take(n_train_samples).batch(batch_size=BATCH_SIZE)
# Skip over the training batches and take the validation batches
val_ds = train_tf.skip(n_train_samples).take(n_val_samples).batch(batch_size=BATCH_SIZE)
# Confirm the number of batches in each
print (f' There are {len(train_ds)} training batches.')
print (f' There are {len(val_ds)} validation batches.')

 There are 40 training batches.
 There are 14 validation batches.


In [52]:
# How many samples in the test data
n_test_samples = len(test_tf)
n_test_samples


726

In [53]:
# Put the test data into batches also
test_ds = test_tf.batch(batch_size = BATCH_SIZE)
# How many batches
print (f' There are {len(test_ds)} testing batches.')

 There are 23 testing batches.


In [55]:
# Save training dataset object
train_ds_fpath = FPATHS['Data']['tf']['train_tf']
tf.data.Dataset.save(train_ds, train_ds_fpath)
# Save testing dataset object
test_ds_fpath = FPATHS['Data']['tf']['test_tf']
tf.data.Dataset.save(test_ds, test_ds_fpath)


In [57]:
# Define classes
# classes = np.unique(example_y)
encoder = joblib.load(FPATHS['Data']['ml']['label_encoder'])
classes = encoder.classes_
classes



array(['Bad Review', 'Good Review'], dtype=object)

In [61]:
FPATHS['Models']['grid']

'Models/gridsearch_89.joblib'

In [ ]:
fpath_gridsearch = FPATHS['Models']['ml']
joblib.dump(Gridsearch_full, fpath_Gridsearch_full)